# Multivariate Fuzzy C-means method: Implementation

## Equations

### $J= \sum_{i=1}^{c} \sum_{k=1}^{n} \sum_{j=1}^{p} \left(u_{ijk} \right)^{m} d_{ijk}$ - Objective function to minimize.

### $d_{ijk} = \left(x_{jk} - y_{ij} \right)^{2}$ - euclidian distance squared.

### $ y_{ij} = \frac{\sum_{k=1}^{n} \left(u_{ijk} \right)^{m} x_{jk}} {\sum_{k=1}^{n} \left(u_{ijk}\right)^{m}}$ - prototype coordinate of a given cluster in feature j.

### $ u_{ijk} =  \left[\sum_{h=1}^{c}\sum_{l=1}^{p} \left(\frac{d_{ijk}}{d_{hlk}}\right)^{(1/(m-1))}  \right]^{-1} $ - membership degree of pattern k in cluster $C_{i}$ on the feature j.

### $\delta_{ik} = \sum_{j=1}^{p} u_{ijk}$ - represents an aggregation measure for all the p features.

## Constraints:

### - $u_{ijk} \in [0, 1]$ for all i, j and k;
### - $0 < \sum_{j=1}^{p} \sum_{k=1}^{n} u_{ijk} < n$ for all i and
### - $\sum_{i=1}^{c}\sum_{j=1}^{p}u_{ijk} = 1$ for all k.

## Importando bibliotecas

In [21]:
import numpy as np
import pandas as pd
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
import warnings
warnings.filterwarnings("ignore")

## Tratamento dos dados

In [22]:
def gerar_configuracao(mu_list, sigma_list, tamanhos, config_id):
    dfs = []
    for i, (mu, sigma2, n) in enumerate(zip(mu_list, sigma_list, tamanhos)):
        Sigma = np.diag(sigma2)
        data = np.random.multivariate_normal(mu, Sigma, n)
        df = pd.DataFrame(data, columns=["x1", "x2"])
        df["class"] = i + 1
        dfs.append(df)
    df_config = pd.concat(dfs, ignore_index=True)
    df_config["config"] = config_id
    return df_config

np.random.seed(42)  # reprodutibilidade

# -------------------------------
# Configuração 1
mu_1 = [[5, 0], [15, 5], [18, 14]]
sigma2_1 = [[81, 9], [9, 100], [25, 36]]
n1 = [200, 100, 50]
df1 = gerar_configuracao(mu_1, sigma2_1, n1, config_id=1)
l1 = "Classes elípticas de tamanhos diferentes"

# -------------------------------
# Configuração 2
mu_2 = [[0, 0], [30, 0], [12, 25]]
sigma2_2 = [[100, 100], [49, 49], [16, 16]]
n2 = [200, 100, 50]
df2 = gerar_configuracao(mu_2, sigma2_2, n2, config_id=2)
l2 = "Classes esféricas de tamanhos diferentes"

# -------------------------------
# Configuração 3
mu_3 = [[0, 0], [15, 5], [15, -5]]
sigma2_3 = [[100, 4], [100, 4], [100, 4]]
n3 = [100, 100, 100]
df3 = gerar_configuracao(mu_3, sigma2_3, n3, config_id=3)
l3 = "Classes elípticas de tamanhos iguais"

# -------------------------------
# Configuração 4
mu_4 = [[0, 0], [15, 0], [-15, 0]]
sigma2_4 = [[16, 16], [16, 16], [16, 16]]
n4 = [100, 100, 100]
df4 = gerar_configuracao(mu_4, sigma2_4, n4, config_id=4)
l4 = "Classes elípticas de tamanhos iguais"

# -------------------------------
# Configuração 5
mu_5 = [[5, 0], [15, 5], [10, -7], [3, 15]]
sigma2_5 = [[81, 9], [9, 100], [49, 16], [25, 25]]
n5 = [50, 50, 50, 50]
df5 = gerar_configuracao(mu_5, sigma2_5, n5, config_id=5)
l5 = "3 classes elípticas e 1 esférica"

# -------------------------------
# Configuração 6
mu_6 = [[5, 0], [15, 5], [12, -12], [7, 17]]
sigma2_6 = [[81, 9], [9, 100], [16, 16], [25, 25]]
n6 = [50, 50, 50, 50]
df6 = gerar_configuracao(mu_6, sigma2_6, n6, config_id=6)
l6 = "2 classes elípticas e 2 esféricas"

# -------------------------------
# Configuração 7
mu_7 = [[0, 0], [18, 0], [-18, 0], [0, -12]]
sigma2_7 = [[12, 12], [20, 20], [16, 16], [81, 20]]
n7 = [50, 50, 50, 50]
df7 = gerar_configuracao(mu_7, sigma2_7, n7, config_id=7)
l7 = "1 classe elíptica e 3 esféricas"

In [23]:
class MFCM():
    def __init__(self, c, X, m):
        self.c = c
        self.n = X.shape[0]
        self.p = X.shape[1]
        self.m = m
    
    def initialize_u(self):
        return np.random.dirichlet(alpha=np.ones(self.c * self.p),
                                   size=self.n).reshape(self.n, self.c, self.p)
    
    def initialize_gamma(self):
        return np.ones((self.n, self.c, self.p))
    
    def find_centroides(self, X, U, Gamma):
        u_gamma = U / np.fmax(Gamma, np.finfo(np.float64).eps) # U / Gamma
        u_gamma_m = u_gamma ** self.m # (U / Gamma)^m
        
        numerador = u_gamma_m * X[:, np.newaxis, :] # (U / Gamma)^m * x_jk
        numerador_sum = np.sum(numerador, axis=0) # somatório em k
        
        denominador_sum = np.sum(u_gamma_m, axis=0) # somatório em k
        denominador_sum = np.fmax(denominador_sum, np.finfo(np.float64).eps) # impede que seja por zero
        
        centroides = numerador_sum / denominador_sum  
        return centroides
    
    def get_distances(self, X, V):
        return (X[:, np.newaxis, :] - V[np.newaxis, :, :]) ** 2

    def update_u(self, D, Gamma):
        P = 1.0 / (self.m - 1)  # expoente de d: 1 / (m-1)
        P_gamma = self.m * P    # expoente do gamma: m / (m-1)
        
        d_inverso = 1.0 / np.fmax(D, np.finfo(np.float64).eps) # d_ijk^(-1)
        d_elevado = d_inverso ** P # ((d_ijk)^(-1)) ^ P
        Gamma_potencia = Gamma ** P_gamma # (gamma_ijk)^(m / (m-1))
        numerador = d_elevado * Gamma_potencia # (N, C, P)
        
        # somatório feito sobre clusters (axis=1) e variáveis (axis=2)
        denominador = np.sum(numerador, axis=(1, 2), keepdims=True)
        
        U = numerador / denominador
        U = U / np.sum(U, axis=(1, 2), keepdims=True) # normalização final
        
        return U
    
    def update_gamma(self, D, U):
        expoente = 1.0 / (self.m + 1)
        
        d = np.fmax(D, np.finfo(np.float64).eps) # d_ijk
        mult = (U ** self.m) * d # [(u_ijk)^m * d_ijk]
        numerador = mult ** expoente # [(u_ijk)^m * d_ijk] ^ (1/(m+1))
        
        den = numerador * U # [(u_ijk)^m * d_ijk]^(1/(m+1)) * u_ijk
        # somatório feito sobre clusters (axis=1) e variáveis (axis=2)
        denominador = np.sum(den, axis=(1, 2), keepdims=True)
        denominador = np.fmax(denominador, np.finfo(np.float64).eps)
        
        gamma = numerador / denominador
        
        return gamma
    
def mfcm_run(dados, num_clusters, m=2, max_iter=10**3, epsilon=1e-5):
    mfcm = MFCM(c=num_clusters, X=dados, m=m) # create the MFCM object
    
    U = mfcm.initialize_u() # initialize the membership matrix
    Gamma = mfcm.initialize_gamma() # initialize the weight matrix
    
    for _ in range(max_iter):
        centroids = mfcm.find_centroides(dados, U, Gamma)
        D = mfcm.get_distances(dados, centroids)
        new_U = mfcm.update_u(D, Gamma)
        Gamma = mfcm.update_gamma(U, D)
        if np.linalg.norm(U - new_U) < epsilon:
            break
        U = new_U

    Delta = np.sum(Gamma * U, axis=2) 
    
    return centroids, U, Delta

def monte_carlo_simulation(dados, labels, num_clusters, num_trials):
    results_ari = []
    results_ami = []
    for _ in range(num_trials):
        centroids, U, Delta = mfcm_run(dados, num_clusters)
        predicted_labels = np.argmax(Delta, axis=1)
        ari = adjusted_rand_score(labels, predicted_labels)
        ami = adjusted_mutual_info_score(labels, predicted_labels)
        if ari > 0.4:
            results_ari.append(ari)
        if ami > 0.4:
            results_ami.append(ami)
    mean_ari = np.mean(results_ari)
    std_ari = np.std(results_ari)
    mean_ami = np.mean(results_ami)
    std_ami = np.std(results_ami)
    return mean_ari, std_ari, mean_ami, std_ami

In [24]:
np.random.seed(42)

i = 1
for df in [df1, df2, df3, df4, df5, df6, df7]:
    if i == 5 or i == 6 or i == 7:
        num_clusters = 4
    else:
        num_clusters = 3
    df.drop("config", axis=1, inplace=True)
    if i == 5 or i == 6 or i == 7:
        df["class"].replace({1: 0, 2: 1, 3: 2, 4: 3}, inplace=True)
    else: 
        df["class"].replace({1: 0, 2: 1, 3: 2}, inplace=True)
    labels = df["class"].values
    df.drop("class", axis=1, inplace=True)
    dados = df.to_numpy()
    num_trials = 100
    mean_ari, std_ari, mean_ami, std_ami = monte_carlo_simulation(dados, labels, num_clusters, num_trials)

    print(f"Monte Carlo MFCM Clustering Results for Config {i}")
    print(f"Mean Adjusted Rand Index: {mean_ari:.4f}")
    print(f"Standard Deviation of Adjusted Rand Index: {std_ari:.4f}")
    print("")
    print(f"Mean Adjusted Mutual Information: {mean_ami:.4f}")
    print(f"Standard Deviation of Adjusted Mutual Information: {std_ami:.4f}")
    print("\n")
    i += 1

Monte Carlo MFCM Clustering Results for Config 1
Mean Adjusted Rand Index: 0.4283
Standard Deviation of Adjusted Rand Index: 0.0129

Mean Adjusted Mutual Information: nan
Standard Deviation of Adjusted Mutual Information: nan


Monte Carlo MFCM Clustering Results for Config 2
Mean Adjusted Rand Index: 0.5455
Standard Deviation of Adjusted Rand Index: 0.0905

Mean Adjusted Mutual Information: 0.5100
Standard Deviation of Adjusted Mutual Information: 0.0717


Monte Carlo MFCM Clustering Results for Config 3
Mean Adjusted Rand Index: nan
Standard Deviation of Adjusted Rand Index: nan

Mean Adjusted Mutual Information: nan
Standard Deviation of Adjusted Mutual Information: nan


Monte Carlo MFCM Clustering Results for Config 4
Mean Adjusted Rand Index: 0.4994
Standard Deviation of Adjusted Rand Index: 0.0806

Mean Adjusted Mutual Information: 0.5583
Standard Deviation of Adjusted Mutual Information: 0.0621


Monte Carlo MFCM Clustering Results for Config 5
Mean Adjusted Rand Index: 0.4271
